[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/porterjenkins/byu-cs474/blob/master/lab4_unets.ipynb)

# Cancer Detection and UNet




In this lab, we will learn on a dataset for a real-world task. We will be performing cancer detection: given a photo of a slide of cells, identify the areas of cancerous cells. We will be performing image segmentation: classifying each pixel as cancerous or non-cancerous.

You will create a large network, using the U-net architecture described below. The main goal of this lab is for you to understand how to create complex network architectures capable of learning difficult tasks.

---
## Grading Criteria


* 20% Completed dataset analysis
* 20% Correct training and validation functions
* 10% Proper creation, training, and validation of smallest-possible network for this task
* 40% Proper creation, training, and validation of the U-net architecture
* 10% Reasonable validation results achieved with the U-net architecture


---

## Lab Setup

In [ ]:
import os
import gzip
import tarfile
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, utils, datasets

import shutil
from pathlib import Path

Set the global variable `device = torch.device("cpu")`. You can change this to `cuda` once you know everything is working, that way you don't run out of your GPU resources on Colab.

Remember to use a CPU runtime when not using the GPU. Colab has strict limits on how long you can use a GPU runtime.

In [ ]:
device = torch.device("cuda")

WARNING: You may run into an error that says `RuntimeError: CUDA out of memory.`

If you get this error immediately when training, then your batch is probably larger than what the GPU is capable of. You can solve this problem by adjusting the image size or the batch size and then restarting the runtime.
If you get this error after training for multiple steps then you probably are not freeing up computation graph, i.e. you are storing `loss` instead of `loss.item()`.

In [ ]:
class CancerDataset(Dataset):
  def __init__(self, root, download=True, size=512, train=True):
    if download and not os.path.exists(os.path.join(root, 'cancer_data')):
      datasets.utils.download_url('http://liftothers.org/cancer_data.tar.gz', root, 'cancer_data.tar.gz', None)
      self.extract_gzip(os.path.join(root, 'cancer_data.tar.gz'))
      self.extract_tar(os.path.join(root, 'cancer_data.tar'))

    postfix = 'train' if train else 'test'
    root = os.path.join(root, 'cancer_data', 'cancer_data')
    self.dataset_folder = datasets.ImageFolder(os.path.join(root, 'inputs_' + postfix) ,transform = transforms.Compose([transforms.Resize(size),transforms.ToTensor()]))
    self.label_folder = datasets.ImageFolder(os.path.join(root, 'outputs_' + postfix) ,transform = transforms.Compose([transforms.Resize(size),transforms.ToTensor()]))

  @staticmethod
  def extract_gzip(gzip_path, remove_finished=False):
    print('Extracting {}'.format(gzip_path))
    with open(gzip_path.replace('.gz', ''), 'wb') as out_f, gzip.GzipFile(gzip_path) as zip_f:
      out_f.write(zip_f.read())
    if remove_finished:
      os.unlink(gzip_path)

  @staticmethod
  def extract_tar(tar_path):
    print('Untarring {}'.format(tar_path))
    z = tarfile.TarFile(tar_path)
    z.extractall(tar_path.replace('.tar', ''))

  def __getitem__(self,index):
    img = self.dataset_folder[index]
    label = self.label_folder[index]
    return img[0],label[0][0]

  def __len__(self):
    return len(self.dataset_folder)

---

# Part 1: Analyzing the Dataset

First things first.
Let's take a look at the dataset.

*Note: This may take a while to download; it may be quicker to upload it to Colab from your machine.*

In [ ]:
train_dataset = CancerDataset("/tmp/Datasets/cancer", train=True)
val_dataset = CancerDataset("/tmp/Datasets/cancer", train=False)

Print out the lengths of your train and validation datasets, so you know how much data you are working with.

In [ ]:
print(len(train_dataset))
print(len(val_dataset))

Now you should examine a single data instance from the dataset.
We'll use pos_test_000072.png (`val_dataset[172]`) as an example throughout training.
Collect the image and the cancer labels from pos_test_000072.png (i.e. `val_x, val_y = val_dataset[172]`).

In [ ]:
val_x, val_y = val_dataset[172]

Let's analyze the `x` data first. Print out the shape and dtype of `val_x` below.

In [ ]:
print(val_x.shape)
print(val_x.dtype)

`x` has 3 channels, which likely means that `x` is an RGB image (which it is).
Since RGB images can be values between [0, 255] or [0, 1] we should print out the min and max values of x to see which.
Print out the min and max values of x below.

In [ ]:
print(val_x.min())
print(val_x.max())

The values are between [0, 1], which is good because that means we don't have do any normalization.

Now implement a `plot_image()` function which should plot an `x` variable.
You can use the `plt.imshow()` function to plot an RGB image.

Plot `val_x` and validate that it looks the same as the image file in "tmp/Datasets/cancer/cancer_data/cancer_data/inputs_test/0/pos_test_000072.png" in the dataset.

*Note: Per PyTorch convention an image has its channels at the 0th dimension (ignoring the batch dimension), but matplotlib expects the channel dimension at the 2nd dimension. You will need to permute the dimensions the plot it correctly*.

In [ ]:
def plot_image(x):
    plt.imshow(x.permute(1, 2, 0))
    plt.show()

plot_image(val_x)

Now let's have a look at the cancer labels `y`; print out the shape and dtype of `val_y`.

In [ ]:
print(val_y.shape)
print(val_y.dtype)

`y` has 1 channel (but it is squeezed so there is no channel dimension).
This most likely means that `y` is a grayscale image (which it is).

Implement `plot_cancer()` which should plot a `y` variable. You can use the `plt.imshow()` function to plot a grayscale image (pass in the `cmap="gray"` argument to make the image grayscale).

Plot `val_y` and validate that it looks like the label image "tmp/Datasets/cancer/cancer_data/cancer_data/outputs_test/0/pos_test_000072.png" in the dataset.

In [ ]:
def plot_cancer(y):
    plt.imshow(y, cmap="gray")
    plt.show()

plot_cancer(val_y)

The cancer is white which means that our cancerous label is 1 and our noncancerous label is 0.
Interestingly, the dtype of `y` is a float, but we were expecting cancer labels (i.e. cancerous vs noncancerous).
Use `torch.unique()` below to see all the unique values of `y`.

In [ ]:
print(torch.unique(val_y))

While 0 and 1 are in `y`, we have other values inbetween. For classification with cross entropy, we need `y` to be integers only. The original image was only black and white, so these non-integer values are due to the image being resized when it was loaded in. We need to think about the dataset and what we want to accomplish to determine how to treat these values: do we round, truncate, or ceiling?

Because this is due to the resizing, these values only occur on the outer edge of the cancer label blobs. For this task, we want to identify regions with cancer, but we don't really need pixel accuracy - we just want the doctors to know to look there for cancer. So, we can truncate these non-one values by calling `y.long()`.

In [ ]:
val_y.long()

You should also add a batch dimension to `val_x` and put it on the `device` so we can pass it through the network later.

In [ ]:
val_x = val_x.unsqueeze(0).to(device)
print(val_x.shape)

Now retrieve some baseline accuracy information about the train and validation datasets.
- How many images have cancerous cells in the dataset
- On average how likely is a pixel going to be cancerous

Print this information below.

In [ ]:
train_num_cancerous = 0
train_cancerous_pixels = 0
val_num_cancerous = 0
val_cancerous_pixels = 0

for i in range(len(train_dataset)):
    label = train_dataset.label_folder[i][0][0]
    train_num_cancerous += 1 if label.long().max() == 1 else 0
    train_cancerous_pixels += label.long().sum()

for i in range(len(val_dataset)):
    label = val_dataset.label_folder[i][0][0]
    val_num_cancerous += 1 if label.long().max() == 1 else 0
    val_cancerous_pixels += label.long().sum()


print(f"{train_num_cancerous} / {len(train_dataset)} items in the training set include cancerous cells")
print(f"{train_cancerous_pixels / len(train_dataset) / 512 / 512 * 100:.2f}% of pixels in the training set are labeled as cancerous")
print(f"{val_num_cancerous} / {len(val_dataset)} items in the validation set include cancerous cells")
print(f"{val_cancerous_pixels / len(val_dataset) / 512 / 512 * 100:.2f}% of pixels in the validation set are labeled as cancerous")

This dataset is very skewed towards non-cancerous data. This makes sense: most cells are non-cancerous, and images that contain cancer will have non-cancerous areas. For our model to be successful, we need to get higher than this baseline accuracy, so our validation accuracy should be over 90%.

---

## Network outputs



Now that we understand our input images and our label images, what should our network's predictions look like?

Because we are doing pixelwise classification, we want to output logits (unnormalized probabilities) for each pixel in our input. So the spatial dimension, i.e. HxW, of the network's output will be the same as those of our input.

#### Loss
If you recall from lab 1, when we implemented cross entropy, our predicted logits had a shape of `(B, K)`, where `K` was the number of classes, and our target classes had a shape of `(B,)`, which was used to index into our predicted logits.
This will work similarly for pixelwise cross entropy, the predicted logits $\hat y$ will have a shape of `(B, K, H, W)`, where `K=2` for our dataset, and the target $y$ will have a shape of `(B, H, W)`.
You can use PyTorch's `F.cross_entropy()` function to compute the cross entropy loss.

#### Accuracy
Implement an accuracy function which compute the pixelwise accuracy between $\hat y$ and $y$.

*Note: Don't forget to use `torch.no_grad()` and since we are just interested in number, return the accuracy as a number, using `.item()`.*

In [ ]:
def cancer_detection_accuracy(y_hat, y):
    with torch.no_grad():
        y = y.to(device)
        return (y_hat.argmax(dim=1) == y).float().mean().item()

Now create a test function for `cancer_detection_accuracy`. Your function should:
- Call `cancer_detection_accuracy()` twice, with two different predictions and accuracies.
- Be simple, but nontrivial. Don't compare all (non)cancerous predictions with all (non)cancerous targets.

Don't forget to add a batch dimension, even if it is just a batch size of 1.

*Note: We recommend you create small images since they are easier to visualize and you have different values for each dimension, except for maybe H and W, because `B, C, H, W = 2, 2, 2, 2` can be harder to understand than `B, C, H, W = 1, 2, 3, 3`.*

In [ ]:
def test_cancer_detection_accuracy():
    y_hat = torch.tensor([[
        [
            [1, 1, 1],
            [0.3, 0.3, 0.3],
            [0.6, 0.6, 0.6],
        ],
        [
            [0, 0, 0],
            [0.7, 0.7, 0.7],
            [0.4, 0.4, 0.4],
        ],
    ]])

    y = torch.tensor([
        [
            [0,1,0],
            [1,0,1],
            [0,1,0]
        ]
    ])
    assert math.isclose(cancer_detection_accuracy(y_hat, y), 2/3, rel_tol=1e-6)

test_cancer_detection_accuracy()

---

## Dataloaders



Now that we have the datasets, put them into dataloaders for training. Define a batch size (should be pretty small because we'll use a huge network). You should always shuffle your training dataset, but this time it is even more important because the dataset has all negative examples first. If you don't shuffle, your network will learn to predict all pixels as non-cancerous.

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4)

---

# Part 2: Training and Validation



Now that we understand the data, we'll create training and validation functions BEFORE creating our network.

We'll start with validation, since its simpler. Create a validation function that takes in a network and validation dataloader and returns the mean cross-entropy loss (just the value, not the tensor) and pixel-wise accuracy over the dataset. Don't forget to use `torch.no_grad()`.

In [ ]:
def validation(net, val_loader):
    loss = 0
    accuracy = 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            y_hat = net(x)
            loss += F.cross_entropy(y_hat, y.long())
            accuracy += cancer_detection_accuracy(y_hat, y)
    return loss / len(val_loader), accuracy / len(val_loader)

Because this is a very visual task, we also want to see how our model's predictions change over time. We'll use a single image: `val_dataset[172]` and store our model's prediction on it each time we perform validation. At the end, we can print out the images to see how our model improves over time. Create a function that takes in an image tensor of shape `(1, C, H, W)` (batch size of 1) and returns a prediction image of shape `(H, W)`, i.e remove the batch dimension and argmax the channel dimension to get the pixel predictions. Don't forget to use `torch.no_grad()`.

In [ ]:
def get_prediction(net, x):
    with torch.no_grad():
        return net(x).argmax(dim=1).squeeze(0)

Create a function(s) to plot your training/validation losses on the same plot and your training/validation accuracies on the same plot.
Label your axes and add a legend to make your plots legible.

In [ ]:
def plot_losses(train_losses, val_losses, train_accs, val_accs, log_interval, val_interval):
    import torch

    def to_float_list(x):
        out = []
        for t in x:
            if torch.is_tensor(t):
                out.append(t.cpu().item())
            elif isinstance(t, tuple):
                out.append(float(t[0]))
            else:
                out.append(float(t))
        return out

    train_losses = to_float_list(train_losses)
    val_losses   = to_float_list(val_losses)
    train_accs   = to_float_list(train_accs)
    val_accs     = to_float_list(val_accs)

    train_steps = [i * log_interval for i in range(len(train_losses))]
    val_steps = [i * val_interval for i in range(len(val_losses))]

    plt.figure(figsize=(10, 5))
    plt.plot(train_steps, train_losses, label="Training Loss")
    plt.plot(val_steps, val_losses, label="Validation Loss")
    plt.xlabel("Steps")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 5))
    plt.plot(train_steps, train_accs, label="Training Accuracy")
    plt.plot(val_steps, val_accs, label="Validation Accuracy")
    plt.xlabel("Steps")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()

Create another function to plot 5 image predictions.

In [ ]:
def plot_predictions(val_images):
    plt.figure(figsize = (10, 4))
    for i in range(len(val_images)):
        img = val_images[i].detach().cpu().squeeze().numpy()

        plt.subplot(1, len(val_images), i + 1)
        plt.imshow(img, cmap="gray")
        plt.axis("off")
    plt.show()

---

Now create your training function. This should be similar to what we've done in lab 3, but remember that your data and labels are different sizes from lab 3's dataset.

Store your training accuracy and losses every `log_interval` steps and validate every `val_interval` steps. Also store 5 predictions of `val_x` dispersed evenly throughout training.
Return the statistics you've gathered (losses, accuracies, and val image predictions).

In [ ]:
def train(net, optimizer, train_dataloader, val_dataloader, n_minibatch_steps, log_interval, val_interval, val_im_delay, val_x):
    # Train net on the training data, keeping track of loss, accuracy, and predictions on val_x
    # Return losses and accuracies on both training and validation datasets, and 5 images
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []

    val_images = []
    checkpoints = [int(round(s)) for s in np.linspace(val_im_delay, n_minibatch_steps - 1, 5)]

    step = 0
    while step < n_minibatch_steps:
        for x, y in train_dataloader:
            if step >= n_minibatch_steps:
                break
            x, y = x.to(device), y.to(device)
            y_hat = net(x)
            loss = F.cross_entropy(y_hat, y.long())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if step % log_interval == 0:
                train_losses.append(loss.item())
                train_accs.append(cancer_detection_accuracy(y_hat, y))

            if step % val_interval == 0:
                val_loss, val_acc = validation(net, val_dataloader)
                val_losses.append(val_loss)
                val_accs.append(val_acc)

            if step in checkpoints:
                val_images.append(get_prediction(net, val_x))
            step += 1

    return train_losses, train_accs, val_losses, val_accs, val_images

---

# Part 3: Training Smallest-Possible Network



While we want to use the UNet on this problem, we should make sure our training and validation runs with a smaller network. Doing this first makes sure that we can focus on bugs in the training and validation loop before focusing on debugging the network itself. And this makes sure that we understand what our inputs and outputs should be.

Before reading further, answer the following question:

What is the smallest-possible convolutional network that can solve this task?

In [ ]:
# CREATE SMALLEST NETWORK
small_net = nn.Conv2d(3, 2, kernel_size=1).to(device)

---

You should have created a one-layer network with 3 in-channels and 2 out-channels, with a 1x1 convolution. This network obviously won't learn well, but it will be useful for quickly testing and debugging our training and validation functions.

In [ ]:
# Create an optimizer for your network
small_optimizer = optim.Adam(small_net.parameters(), lr=0.001)

# Train for a few training steps: enough to gather some stats
train_losses, train_accs, val_losses, val_accs, val_images = train(
    small_net,
    small_optimizer,
    train_loader,
    val_loader,
    16,
    1,
    3,
    0,
    val_dataset[172][0].unsqueeze(0).to(device)
)


In [ ]:
# Create your loss and accuracy graphs, and print out the 5 validation images:
# We do not expect this network to have learned well.

plot_losses(train_losses, val_losses, train_accs, val_accs, 1, 3)
plot_predictions(val_images)

In [ ]:
# We don't need this network anymore, so we'll delete it to free up GPU space
del small_net, small_optimizer, train_losses, train_accs, val_losses, val_accs, val_images

---

# Part 4: Implementing the UNet

Use the “Deep Convolution U-Net” from this paper: [(U-Net: Convolutional Networks for Biomedical Image Segmentation)](https://arxiv.org/pdf/1505.04597.pdf)

![(Figure 1)](https://lh3.googleusercontent.com/qnHiB3B2KRxC3NjiSDtY08_DgDGTDsHcO6PP53oNRuct-p2QXCR-gyLkDveO850F2tTAhIOPC5Ha06NP9xq1JPsVAHlQ5UXA5V-9zkUrJHGhP_MNHFoRGnjBz1vn1p8P2rMWhlAb6HQ=w2400)
<!-- <img src="https://lh3.googleusercontent.com/qnHiB3B2KRxC3NjiSDtY08_DgDGTDsHcO6PP53oNRuct-p2QXCR-gyLkDveO850F2tTAhIOPC5Ha06NP9xq1JPsVAHlQ5UXA5V-9zkUrJHGhP_MNHFoRGnjBz1vn1p8P2rMWhlAb6HQ=w2400" width="1600" height="1200"/> -->

You will implement the UNet with some simplifying adjustments.
The original UNet does not pad their convolutions, so their output is smaller than their input.
Your input and output are the same size so you should pad your convolutions.
The only time height and width should change are from "max pool" and the "up-conv".
This means you no longer need to "crop" the cross connections, since both sides of the U should have the same spatial dimensions.

This is a large complex network structure and due to the cross connections, you can't just stick a bunch of layers into `nn.Sequential`.
This network does have a clear repeating pattern of blocks that move down (left side of U) and blocks that move up (right side of U).
Just like how we use functions to made our code more modular, we'll create two modules to capture that repeating structure to simplify our final network.

Implement `DownBlock` (left side of U): The `DownBlock` incorporates 2 sequential convolution layers (with nonlinearities) followed by a downsampling (`nn.MaxPool2d`).
It should return the downsampled image and the cross connection output.
You should NOT use `copy` or `clone` on the cross connection output to do the "copy and crop" operation; just return the same tensor you passed into `MaxPool2D` layer.

In [ ]:
# You are welcome (and encouraged) to use the built-in batch normalization, nn.BatchNorm2d, and dropout layers, nn.Dropout.

class DownBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
        )
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

    def forward(self, x):
        cross_output = self.conv(x)
        return self.pool(cross_output), cross_output

Implement `UpBlock` (right side of U): The `UpBlock` incorporates an upsampling (i.e. `nn.ConvTranspose2d`) followed by 2 sequential convolution layers (with nonlinearities). Its input should be a downsampled image, `x`, and the cross connection output, `cross_output`, from a `DownBlock`.

In [ ]:
class UpBlock(nn.Module):
    def __init__(self, in_channels, out_channels) -> None:
        super().__init__()
        self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )

    def forward(self, x, cross_output):
        x = self.up(x)
        x = torch.cat((x, cross_output), dim=1)
        return self.conv(x)

Test below that your `DownBlock` and `UpBlock` work correctly.
Create a function `test_updown_block()` which instantiates a `DownBlock` and `UpBlock` and passes in a dummy image `x = torch.zeros((2, 4, 16, 16))`.
The downsampled output of your `DownBlock` should be passed through the provided `mid_conv` layer (representing the bottom layer in the UNet) before being passed to the `UpBlock`.
You will assert that the the cross-connection output of your `DownBlock` and the output of your `UpBlock` have the shape of `(2, 8, 16, 16)`.
Remember to use `torch.no_grad()`.

In [ ]:
def test_updown_block():
    mid_conv = nn.Sequential(
        nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1),
        nn.ReLU()
    )
    down = DownBlock(4, 8)
    up = UpBlock(16, 8)

    x = torch.zeros((2, 4, 16, 16))

    with torch.no_grad():
        down_out, cross_out = down(x)
        mid_out = mid_conv(down_out)
        up_out = up(mid_out, cross_out)

    assert down_out.shape == (2, 8, 8, 8)
    assert up_out.shape == (2, 8, 16, 16)

test_updown_block()

Implement the `UNet` using your `UpBlock` and `DownBlock` modules.
You will additionally need to add layers for the bottom of the U and the final output layer.
You do not need to use the same number of channels as the UNet paper. It is recommended you use fewer channels for faster training and less memory consumption. `[8, 16, 32, 64]` are a good choice for channel sizes.

*Note: If you want to store `UpBlock`s and `DownBlock`s in a list then use the `nn.ModuleList()` which is a module that operates like a list, but exposes the modules to PyTorch.*

In [ ]:
class UNet(nn.Module):
    def __init__(self, channels=(8, 16, 32, 64)) -> None:
        super().__init__()

        self.down_modules = nn.ModuleList()
        self.down_modules.append(DownBlock(3, channels[0]))
        for i in range(len(channels) - 1):
            self.down_modules.append(DownBlock(channels[i], channels[i+1]))

        self.mid_module = nn.Sequential(
            nn.Conv2d(channels[-1], channels[-1]*2, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(channels[-1]*2, channels[-1]*2, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )

        self.up_modules = nn.ModuleList()
        self.up_modules.append(UpBlock(channels[-1]*2, channels[-1]))
        for i in range(len(channels) - 1, 0, -1):
            self.up_modules.append(UpBlock(channels[i], channels[i-1]))
        self.output_module = nn.Conv2d(channels[0], 2, kernel_size=1, stride=1)

    def forward(self, x):
        cross_outputs = []
        for i in range(len(self.down_modules)):
            x, cross_output = self.down_modules[i](x)
            cross_outputs.insert(0, cross_output)
        x = self.mid_module(x)
        for i in range(len(self.up_modules)):
            cross_output = cross_outputs[i]
            x = self.up_modules[i](x, cross_output)
        x = self.output_module(x)
        return x

Test `UNet` by making sure that it takes a batch of input images `x` and returns an appropriately-shaped output.

In [ ]:
def test_UNet_shapes():
    unet = UNet()
    x = torch.zeros((2, 3, 512, 512))

    with torch.no_grad():
        y = unet(x)
        assert y.shape == (2, 2, 512, 512)

test_UNet_shapes()

---

# Training and Testing UNet

Train `UNet` below and get a better than baseline accuracy. For Colab/Kaggle users, we recommend training with a small `UNet` first on the CPU and then scaling up the `UNet` and training on the GPU.

In [ ]:
# Here are some example hyperparameters: feel free to play around with them for better results
n_minibatch_steps = 2000 # number of optimization steps
train_log_interval = 10 # how many steps between saving training loss and accuracy
val_interval = 200 # how many steps between performing the validation loop
val_im_delay = 50 # how many steps before we start storing validation image predictions

# small hyperparameters
# n_minibatch_steps = 16 # number of optimization steps
# train_log_interval = 3 # how many steps between saving training loss and accuracy
# val_interval = 5 # how many steps between performing the validation loop
# val_im_delay = 0 # how many steps before we start storing validation image predictions

# TODO: create UNet and train it, storing statistics from training/validating
unet = UNet().to(device)
# unet = UNet(channels=(2, 4, 8, 16)).to(device)
optimizer = optim.Adam(unet.parameters(), lr=0.001)
train_losses, train_accs, val_losses, val_accs, val_images = train(
    unet,
    optimizer,
    train_loader,
    val_loader,
    n_minibatch_steps,
    train_log_interval,
    val_interval,
    val_im_delay,
    val_dataset[172][0].unsqueeze(0).to(device)
)

In [ ]:
print(get_n_net_params(unet))

Plot your training/validation losses and accuracies.

In [ ]:
plot_losses(train_losses, val_losses, train_accs, val_accs, train_log_interval, val_interval)

Visualize the 5 predictions of `val_x`. Your images should start to look like the `plot_cancer()` image you made above (signs of cancer in the top left corner).

In [ ]:
plot_predictions(val_images)